# Experiment 2: Cross Validation

In [1]:
import os
import sys
import pickle
from glob import glob
from time import time
from tqdm.notebook import tqdm
from collections import Counter

# scikit-learn
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

# data processing
import cv2
import numpy as np
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline

# pytorch
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms, models, utils
torch.manual_seed(0)
print(f'Pytorch version: {torch.__version__}')

# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'This notebook use {device}')

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Pytorch version: 1.6.0
This notebook use cuda:0


In [2]:
# 파일 경로 사용자 정의
class path:
    data = '../input/data'
    train = '../input/data/train'
    train_img = f'{train}/images'
    train_df = f'{train}/train.csv'
    test = '../input/data/eval'
    test_img = f'{test}/images'
    test_df = f'{test}/info.csv'

In [3]:
BATCH_SIZE=16
NUM_WORKERS=2
LEARNING_RATE=1e-4
EPOCHS=3

In [4]:
df = pd.read_csv(f'{path.train}/train_modified.csv')
pd.DataFrame(df['target'].value_counts())

,target
4,4085
3,3660
0,2745
1,2050
16,817
10,817
9,732
15,732
6,549
12,549


## 1. Dataset Definition

In [5]:
class MaskDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def set_transform(self, transform):
        self.transform = transform
    
    def __getitem__(self, index):
        data = self.df.iloc[index]
        target = data.target
        image = Image.open(data.path)
        
        if self.transform:
            image = self.transform(image)
        
        return image, target
        
    def __len__(self):
        return len(self.df)

In [6]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [7]:
train_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, saturation=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
    AddGaussianNoise(0., 1.,)
])

In [8]:
valid_transforms = transforms.Compose([
    transforms.CenterCrop(384),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.548, 0.504, 0.479], std=[0.237, 0.247, 0.246]),
])

## 2. Modeling

In [9]:
model = models.resnet18(pretrained=False)
n_features = model.fc.in_features
model.fc = nn.Linear(n_features, 18)
model = model.cuda()

In [10]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss().to(device)

## 3. Training

In [11]:
def test_eval(model, valid_dataset):
    model.eval()
    with torch.no_grad():
        y_true, y_pred = [], []
        for image, label in tqdm(valid_dataset):
            X = image.view(-1, 3, 224, 224).float().to(device)
            y = label.item()
            _, pred = torch.max(model(X), 1)
            pred = pred.item()
            y_true.append(y)
            y_pred.append(pred)
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        accuracy = accuracy_score(y_true, y_pred)
    model.train()
    return f1, accuracy

In [18]:
def train_model(train, test, model, criterion, optimizer, print_every=1):
    best_accuracy = 0
    for epoch in range(EPOCHS):
        loss_sum = 0
        for images, label in tqdm(train):
            X = images.view(-1, 3, 224, 224).float().to(device)
            y = label.to(device)
            
            y_pred = model(X)
            loss = criterion(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
            loss_sum += loss
          
        if ((epoch%print_every)==0) or (epoch==(EPOCHS-1)):
            loss_avg = loss_sum / len(train)
            f1, accuracy = test_eval(model, test)
            print(f">> epoch:[{epoch+1}/{EPOCHS}] cost:{loss_avg:5.3f} test_accuracy:{accuracy:5.3f} test_f1_score:{f1:5.3f}") 

In [19]:
def cross_validation(df, model, criterion, optimizer, k_folds=5):
    skf = StratifiedKFold(n_splits=5)
    n_iter = 0
    for train_idx, valid_idx in skf.split(df, df.target):
        n_iter += 1
        print(f'>> Cross Validation {n_iter} Starts!')
        train, valid = df.loc[train_idx], df.loc[valid_idx]
        train_dataset, valid_dataset = MaskDataset(train), MaskDataset(valid)
        
        # augmentation 설정
        train_dataset.set_transform(train_transforms)
        valid_dataset.set_transform(valid_transforms)

        # 데이터로더 생성
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
        valid_loader = DataLoader(valid_dataset, shuffle=False)
        
        train_model(train_loader, valid_loader, model, criterion, optimizer)
        print()

In [21]:
df = pd.read_csv(f'{path.train}/train_modified.csv')[['path', 'target']]
cross_validation(df, model, criterion, optimizer)

>> Cross Validation 1 Starts!



>> epoch:[1/3] cost:0.507 test_accuracy:0.678 test_f1_score:0.518



>> epoch:[2/3] cost:0.417 test_accuracy:0.767 test_f1_score:0.621



>> epoch:[3/3] cost:0.338 test_accuracy:0.641 test_f1_score:0.524

>> Cross Validation 2 Starts!



>> epoch:[1/3] cost:0.364 test_accuracy:0.722 test_f1_score:0.614


...


>> epoch:[2/3] cost:0.141 test_accuracy:0.933 test_f1_score:0.891



>> epoch:[3/3] cost:0.124 test_accuracy:0.917 test_f1_score:0.846

>> Cross Validation 5 Starts!



>> epoch:[1/3] cost:0.132 test_accuracy:0.938 test_f1_score:0.916



>> epoch:[2/3] cost:0.109 test_accuracy:0.884 test_f1_score:0.867



>> epoch:[3/3] cost:0.096 test_accuracy:0.898 test_f1_score:0.843

